# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
# Had to ".." my path every time.
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.2000,-175.2000,22.09,94,20,1.03,TO,1680530657
1,1,busselton,-33.6500,115.3333,14.92,87,0,4.25,AU,1680530649
2,2,lavrentiya,65.5833,-171.0000,-15.63,97,100,2.23,RU,1680530658
3,3,thompson,55.7435,-97.8558,-22.91,77,100,3.09,CA,1680530658
4,4,bredasdorp,-34.5322,20.0403,21.86,52,65,7.50,ZA,1680530658


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df.loc[(city_data_df["Max Temp"] >= 18) & (city_data_df["Max Temp"] <= 26)]
filtered_df = filtered_df.loc[filtered_df["Humidity"] <= 60]
filtered_df = filtered_df.loc[filtered_df["Cloudiness"] <= 80]

# Drop any rows with null values
filtered_data_df = filtered_df.dropna()

# Display sample data
filtered_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,bredasdorp,-34.5322,20.0403,21.86,52,65,7.50,ZA,1680530658
7,7,castro,-24.7911,-50.0119,20.31,60,0,1.71,BR,1680530658
19,19,cape town,-33.9258,18.4232,22.84,58,40,3.13,ZA,1680530055
40,40,cidreira,-30.1811,-50.2056,23.87,60,10,3.98,BR,1680530665
112,112,aguilas,37.4063,-1.5829,18.08,46,44,4.83,ES,1680530676
128,128,ourinhos,-22.9789,-49.8706,24.06,57,0,2.43,BR,1680529973
210,210,hirekerur,14.4667,75.3833,25.94,41,13,5.80,IN,1680530683
229,229,san bartolome de tirajana,27.9248,-15.5733,19.26,57,20,12.35,ES,1680530687
258,258,yekaterinogradskaya,43.7635,44.2317,18.05,41,61,3.93,RU,1680530689
261,261,abu dhabi,24.4667,54.3667,24.06,56,0,8.23,AE,1680530351


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\ayelb\AppData\Local\Temp\ipykernel_7340\423860418.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
4,bredasdorp,ZA,-34.5322,20.0403,52,
7,castro,BR,-24.7911,-50.0119,60,
19,cape town,ZA,-33.9258,18.4232,58,
40,cidreira,BR,-30.1811,-50.2056,60,
112,aguilas,ES,37.4063,-1.5829,46,
128,ourinhos,BR,-22.9789,-49.8706,57,
210,hirekerur,IN,14.4667,75.3833,41,
229,san bartolome de tirajana,ES,27.9248,-15.5733,57,
258,yekaterinogradskaya,RU,43.7635,44.2317,41,
261,abu dhabi,AE,24.4667,54.3667,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
    "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: No hotel found
castro - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
aguilas - nearest hotel: No hotel found
ourinhos - nearest hotel: No hotel found
hirekerur - nearest hotel: No hotel found
san bartolome de tirajana - nearest hotel: No hotel found
yekaterinogradskaya - nearest hotel: No hotel found
abu dhabi - nearest hotel: No hotel found
tiznit - nearest hotel: No hotel found
tatawin - nearest hotel: No hotel found
lashio - nearest hotel: No hotel found
nishihara - nearest hotel: No hotel found
port augusta - nearest hotel: No hotel found
bahsuma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,bredasdorp,ZA,-34.5322,20.0403,52,No hotel found
7,castro,BR,-24.7911,-50.0119,60,No hotel found
19,cape town,ZA,-33.9258,18.4232,58,No hotel found
40,cidreira,BR,-30.1811,-50.2056,60,No hotel found
112,aguilas,ES,37.4063,-1.5829,46,No hotel found
128,ourinhos,BR,-22.9789,-49.8706,57,No hotel found
210,hirekerur,IN,14.4667,75.3833,41,No hotel found
229,san bartolome de tirajana,ES,27.9248,-15.5733,57,No hotel found
258,yekaterinogradskaya,RU,43.7635,44.2317,41,No hotel found
261,abu dhabi,AE,24.4667,54.3667,56,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
# %%capture --no-display
# Configure the map plot
overall_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    alpha= 0.5,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map plot
overall_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)